Primero, necesitamos Numba:

In [1]:
import numba


Numba no puede trabajar con:

* Listas o diccionarios.

In [2]:
'''Función que nos genera una población inicial de 2k individuos y 
   recibe como parámetros: La 'k' y el hipercubo al que pertenecen 
   los vectores (o sea, la dimensión 'n' y el intervalo [a,b] al que
   pertenecen cada una de las entradas de los vectores).'''

import numpy as np

@numba.njit
def Generar_Poblacion(k, n, intervalo):
    
   a, b = intervalo

   # Se genera una matriz de tamaño (2k, n) con valores aleatorios
   # uniformes (o sea, una población inicial de 2k individuos donde 
   # cada uno vive en Rn):
   poblacionMatriz = np.random.uniform(a,b, size=(2*k, n))

   return poblacionMatriz # No hacemos la conversión en lista de listas
                          # porque necesitamos de `numba`.


In [3]:
'''Definimos una función de aptitud que recibe como parámetros una
   función de prueba y una población de 2k individuos y retorna una
   lista con las probabilidades de selección (p_i) de todos los 
   individuos de la población. Además de que nos da al mejor individuo
   en la población.'''


@numba.njit
def Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacion):
    
    # Necesitamos el número de individuos de la población:
    numIndividuos = len(poblacion) 
    
    # Generamos un array con las evaluaciones en la función objetivo
    evaluaciones = np.empty(numIndividuos) # Primero generamos un array
                                           # vacío de tamaño adecuado.
    for i in range(numIndividuos):
        evaluaciones[i] = funcion(poblacion[i])

    # Generamos un array de las aptitudes de los individuos en la 
    # población.
    aptitudes = np.exp(1e-9 * (-evaluaciones))

    # Encontramos el índice del mejor individuo (menor evaluación)
    mejorIndice = np.argmax(aptitudes)
    masApto = poblacion[mejorIndice]

    # Obtenemos el 'tamaño del pastel':
    sumaAptitudes = np.sum(aptitudes)

    # Calculamos las probabilidades de selección (p_i) para todos los
    # individuos:
    probabilidades = aptitudes / sumaAptitudes


    return probabilidades, masApto


In [4]:
'''Definimos una función que recibe la lista de probabilidades 
   de selección y nos retorna la que corresponde a la de las 
   probabilidades acumulativas.'''

@numba.njit
def Probas_Acumulativas(probabilidades):

    # Calculamos las probabilidades acumulativas (q_i) para todos los
    # individuos:
    probasAcumulativas = np.cumsum(probabilidades)

    return probasAcumulativas # Retornamos las `probasAcumulativas` como
                              # un arreglo de NumPy (igual es iterable).


Padres mediante el uso de la ruleta:

In [5]:
'''Estas funciones son las que nos ayudan a generar a los padres mediante
   el uso de la ruleta.'''
   

'''La primera es para seleccionar padres (uno) y recibe como parámetro 
   la lista de las probabilidades acumulativas.
   
   Nos retornará el índice en la lista que le corresponde al padre 
   elegido con esa regla.'''

@numba.njit
def Seleccionar_Padre_Ruleta(probasAcumuladas):
   r = np.random.random() # Generamos un número aleatorio entre 0 y 1

   tol=1e-9
   
   # Recorremos las probabilidades acumuladas para seleccionar el padre
   for i in range(len(probasAcumuladas)):
      if r <= probasAcumuladas[i] or np.isclose(r, probasAcumuladas[i], atol=tol):
         return i  # Índice del individuo seleccionado
               
   return -1  # En caso de que no se seleccione, aunque esto no debería suceder.


In [6]:
'''Le segunda es para generar la lista de padres elegidos para 
   reproducirse. Necesitamos 2k nuevos individuos, así que se van a 
   elegir k parejas (cada par de padres produce dos hijos).
   La función recibe como parámetros: la lista de individuos, las
   probabilidades acumulativas y un máximo de intentos para intentar evitar
   la generación de clones (en este caso, vamos a considerar un máximo
   de 100 para cada elección de un segundo padre, pero tenemos la 
   apertura de modificarlo después).'''

@numba.njit
def Generar_Parejas(individuos, probasAcumuladas, maxIntentos=100):
    
    numIndividuos = len(individuos)

    # Validación: si no hay suficientes individuos
    if numIndividuos < 2:
        print("Error: No hay suficientes individuos para generar parejas.")
        return np.empty((0, 2, individuos.shape[1]))
        # Regresamos un array vacío

    # Como len(individuos) = 2k, le pedimos que elija len(individuos)/2 
    # parejas de padres.
    parejasNecesarias = numIndividuos // 2
    
    
    parejas_idxs = np.empty((parejasNecesarias, 2), dtype=np.int32)
    # Creamos un array para los índices de tamaño adecuado.
    # Como queremos parejas, el array tiene dos columnas, la idea es que
    # cada fila del array represente una pareja de padres.

    '''En otras palabras, `parejas_idxs` tendrá la forma:
       [[padre1_idx1, padre2_idx1],
       [padre1_idx2, padre2_idx2],
       [padre1_idx3, padre2_idx3],
       ...
       [padre1_idxN, padre2_idxN]]'''


    for i in range(parejasNecesarias):
        padre1_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)

        intentos = 0
        while True:
            padre2_idx = Seleccionar_Padre_Ruleta(probasAcumuladas)
            if padre1_idx != padre2_idx:
                break  # Padres diferentes, salimos del ciclo
            intentos += 1 # No encontramos un padre distinto, hacemos 
                          # otro intento.
            if intentos >= maxIntentos: # Rebasamos los intentos que
                                        # teníamos permitidos.
                padre2_idx = padre1_idx # Aceptamos el clon.
                break

        parejas_idxs[i, 0] = padre1_idx # Primer padre se asigna en la
                                        # fila `i` de la primer columna.
        parejas_idxs[i, 1] = padre2_idx # Segundo padre se asigna en la
                                        # fila `i` de la segunda columna.

    # Ahora construimos el array de parejas usando los índices generados
    # para almacenar las parejas de individuos.
    parejas = np.empty((parejasNecesarias, 2, individuos.shape[1]), dtype=individuos.dtype)
    
    for i in range(parejasNecesarias):
        parejas[i, 0] = individuos[parejas_idxs[i, 0]]  # Primer padre
        parejas[i, 1] = individuos[parejas_idxs[i, 1]]  # Segundo padre

    
    return parejas


**Ojo:** Vas a tener que volver a definir las funciones de prueba, pero sólo hazlo si ves que de veras mejoraste en los tiempos. Si no, no hagas nada.

Funciones auxiliales (optimizadas con NumPy y Numba):

In [24]:
@numba.njit
def codifica_real(x, n_bit, intervalo):
    """
    Codifica un número real en el intervalo [a,b] utilizando n_bit bits
    y una partición uniforme en [a, b].

    Parámetros:
    x: número real a codificar.
    n_bit: número de bits a utilizar.
    intervalo: (a, b), extremos del intervalo.

    Retorno:
    Arreglo binario que representa al número real x con n_bit bits
    en el intervalo [a, b].
    """
    a, b = intervalo

    # Calcula la precisión de la representación.
    precision = (b - a) / ((2 ** n_bit) - 1)

    # Asegura que el número esté dentro del rango de la partición.
    x = max(a, min(b, x))

    # Calcula el índice del número en la partición.
    index = int((x - a) / precision)

    # Codifica el índice a binario
    if index < 0 or index >= (1 << n_bit):
        raise ValueError(f"Índice fuera del rango representable con {n_bit} bits.")

    x_binario = np.zeros(n_bit, dtype=np.int32)
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1

    return x_binario

@numba.njit
def codifica_vector(vector_reales, n_bit, intervalo):
    """
    Codifica un vector de números reales en un vector de vectores binarios
    utilizando n_bit bits.

    Parámetros:
    vector_reales: Arreglo de números reales a codificar
    n_bit: Número de bits a utilizar para las entradas de nuestro arreglo.
    intervalo: (a, b), extremos del intervalo.

    Retorno:
    Arreglo de arreglos binarios, donde cada sub-arreglo representa un número real
    codificado en binario.
    """
    dim_x = vector_reales.shape[0]

    # Inicializa un arreglo vacío para almacenar los vectores binarios.
    vector_binario = np.empty((dim_x, n_bit), dtype=np.int32)

    for i in range(dim_x):
        vector_binario[i] = codifica_real(vector_reales[i], n_bit, intervalo)

    return vector_binario


In [18]:
'''Codificación de los padres en vectores binarios del estilo
   [[0,1,1,1,...], ..., [1, 1, 1, 0,...]]'''

#@numba.njit
def Padres_Binarios(parejas, nBits, intervalo):

    numParejas = len(parejas)

    # Inicializamos un arreglo vacío para almacenar los vectores binarios.
    parejasBinarias = np.empty((numParejas, 2), dtype=object)

    for i in range(numParejas):
        padre1, padre2 = parejas[i]

        # Codifica cada padre en binario y lo almacena.
        parejasBinarias[i, 0] = codifica_vector(padre1, nBits, intervalo)
        parejasBinarias[i, 1] = codifica_vector(padre2, nBits, intervalo)
    
    return parejasBinarias


In [19]:
'''Veamos un ejemplo de implementación:'''

@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

k = 2
n = 2
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 5

poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"La población inicial fue:\n{poblacionInicial}\n")

pis, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)
print(f"Sus probabilidades de selección fueron:\n{pis}\n")
print(f"El más apto fue:\n{masApto}\n")

qis = Probas_Acumulativas(pis)
print(f"Las probabilidades acumulativas fueron:\n{qis}\n")

parejas = Generar_Parejas(poblacionInicial, qis)
print(f"Las parejas seleccionadas para la reproducción fueron:\n{parejas}\n")

parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres en binario fueron:\n{parejasBinarias}\n")


La población inicial fue:
[[ 1.22693275 -0.34722369]
 [ 1.4109607  -1.4717668 ]
 [-0.98996074  1.86906546]
 [-0.73859517 -0.1861529 ]]

Sus probabilidades de selección fueron:
[0.25000002 0.24999981 0.25000008 0.25000009]

El más apto fue:
[-0.73859517 -0.1861529 ]

Las probabilidades acumulativas fueron:
[0.25000002 0.49999982 0.74999991 1.        ]

Las parejas seleccionadas para la reproducción fueron:
[[[-0.98996074  1.86906546]
  [-0.73859517 -0.1861529 ]]

 [[ 1.4109607  -1.4717668 ]
  [-0.98996074  1.86906546]]]

Los padres en binario fueron:
[[array([[0, 1, 0, 0, 0],
         [1, 1, 1, 0, 1]], dtype=int32)
  array([[0, 1, 0, 0, 1],
         [0, 1, 1, 1, 0]], dtype=int32)]
 [array([[1, 1, 0, 1, 0],
         [0, 0, 1, 0, 0]], dtype=int32)
  array([[0, 1, 0, 0, 0],
         [1, 1, 1, 0, 1]], dtype=int32)]]



**HASTA AQUÍ JALA**

**Intento con otra versión para la codificación de padres.**

In [41]:
@numba.njit
def Padres_Binarios(parejas, nBits, intervalo):
    """
    Codifica parejas de vectores en binario.
    """
    numParejas = parejas.shape[0]
    # Las dimensiones son (numParejas, 2, nBits)
    parejasBinarias = np.empty((numParejas, 2, parejas.shape[1], nBits), dtype=np.int32)

    for i in range(numParejas):
        padre1 = parejas[i, 0]  # Vector 1
        padre2 = parejas[i, 1]  # Vector 2

        # Codifica cada padre en binario y lo almacena.
        parejasBinarias[i, 0] = codifica_vector(padre1, nBits, intervalo)
        parejasBinarias[i, 1] = codifica_vector(padre2, nBits, intervalo)

    return parejasBinarias



In [42]:
'''Veamos un ejemplo de implementación:'''

@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

k = 2
n = 2
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 5

poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"La población inicial fue:\n{poblacionInicial}\n")

pis, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)
print(f"Sus probabilidades de selección fueron:\n{pis}\n")
print(f"El más apto fue:\n{masApto}\n")

qis = Probas_Acumulativas(pis)
print(f"Las probabilidades acumulativas fueron:\n{qis}\n")

parejas = Generar_Parejas(poblacionInicial, qis)
print(f"Las parejas seleccionadas para la reproducción fueron:\n{parejas}\n")

parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres en binario fueron:\n{parejasBinarias}\n")


La población inicial fue:
[[ 0.3570814   1.79795852]
 [ 0.06076734 -0.84707875]
 [ 1.17548119 -1.75578368]
 [-1.56543899 -1.03892264]]

Sus probabilidades de selección fueron:
[0.25000009 0.25000014 0.24999991 0.24999985]

El más apto fue:
[ 0.06076734 -0.84707875]

Las probabilidades acumulativas fueron:
[0.25000009 0.50000023 0.75000015 1.        ]

Las parejas seleccionadas para la reproducción fueron:
[[[ 0.06076734 -0.84707875]
  [-1.56543899 -1.03892264]]

 [[-1.56543899 -1.03892264]
  [ 0.3570814   1.79795852]]]

Los padres en binario fueron:
[[[[0 1 1 1 1]
   [0 1 0 0 1]]

  [[0 0 0 1 1]
   [0 0 1 1 1]]]


 [[[0 0 0 1 1]
   [0 0 1 1 1]]

  [[1 0 0 1 0]
   [1 1 1 0 1]]]]



In [45]:
import numpy as np
import numba

@numba.njit
def cruza_n_puntos_np_numba(padres, n_puntos):
    num_parejas = padres.shape[0]  # Número de parejas de padres
    num_cromosomas = padres[0, 0].shape[0]  # Número de cromosomas por padre (filas)
    num_genes = padres[0, 0].shape[1]  # Número de genes por cromosoma (columnas)

    # Inicializamos un array vacío para almacenar los hijos
    hijos = np.empty((2 * num_parejas, num_cromosomas, num_genes), dtype=np.int32)

    # Iteramos sobre cada pareja de padres
    for pareja_idx in range(num_parejas):
        padre1 = padres[pareja_idx, 0]
        padre2 = padres[pareja_idx, 1]

        # Generamos los hijos de la pareja actual
        hijo1 = np.empty_like(padre1)
        hijo2 = np.empty_like(padre2)

        # Generar n puntos de corte únicos
        puntos_corte = np.random.choice(np.arange(1, num_genes), n_puntos, replace=False)
        puntos_corte.sort()  # Asegurarse de que los puntos de corte están ordenados

        # Iteramos sobre cada cromosoma de los padres
        for cromo_idx in range(num_cromosomas):
            cromo1 = padre1[cromo_idx]
            cromo2 = padre2[cromo_idx]

            # Alternamos segmentos entre los padres
            ultimo_punto = 0
            switch = False

            for punto in np.append(puntos_corte, num_genes):  # Añadimos el último punto
                if switch:
                    hijo1[cromo_idx, ultimo_punto:punto] = cromo2[ultimo_punto:punto]
                    hijo2[cromo_idx, ultimo_punto:punto] = cromo1[ultimo_punto:punto]
                else:
                    hijo1[cromo_idx, ultimo_punto:punto] = cromo1[ultimo_punto:punto]
                    hijo2[cromo_idx, ultimo_punto:punto] = cromo2[ultimo_punto:punto]

                switch = not switch
                ultimo_punto = punto

        # Guardamos los hijos generados en el array de hijos
        hijos[2 * pareja_idx] = hijo1
        hijos[2 * pareja_idx + 1] = hijo2

    return hijos

# Lista de padres que mencionas, ahora en np.array desde el principio
padres = np.array([
    [np.array([[1, 1, 0, 0, 1],
               [0, 1, 0, 0, 0]], dtype=np.int32),
     np.array([[0, 0, 1, 0, 0],
               [0, 1, 1, 1, 0]], dtype=np.int32)],
    [np.array([[1, 0, 0, 0, 1],
               [0, 1, 1, 1, 0]], dtype=np.int32),
     np.array([[0, 0, 1, 0, 0],
               [1, 0, 0, 1, 1]], dtype=np.int32)]
])

# Realizamos la cruza de 2 puntos
n_puntos = 2
hijos_generados = cruza_n_puntos_np_numba(padres, n_puntos)

# Imprimir los hijos generados
for idx, hijo in enumerate(hijos_generados):
    print(f"Hijo {idx + 1}:\n{hijo}\n")


Hijo 1:
[[1 0 0 0 1]
 [0 1 0 0 0]]

Hijo 2:
[[0 1 1 0 0]
 [0 1 1 1 0]]

Hijo 3:
[[1 0 0 0 1]
 [0 1 1 1 0]]

Hijo 4:
[[0 0 1 0 0]
 [1 0 0 1 1]]



In [46]:
'''Veamos un ejemplo de implementación:'''

@numba.njit
def Rosenbrock(x):
    """
    Esta función recibe como parámetro un vector de n entradas, ingresado como un array de NumPy
    y devuelve el escalar que resulta de evaluar la función de Rosenbrock en el vector.
    Tiene un mínimo global en f(1, ..., 1).

    Ejemplo de uso:
      > Rosenbrock(np.array([1, 1]))
      > 0
    """
    n = x.shape[0]  # Obtener el tamaño del array
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

k = 2
n = 2
intervalo = intervaloRosenbrock
funcion = Rosenbrock
nBits = 5
nPuntos = 4

poblacionInicial = Generar_Poblacion(k, n, intervalo)
print(f"La población inicial fue:\n{poblacionInicial}\n")

pis, masApto = Probabilidades_De_Seleccion_Y_Mas_Apto(funcion, poblacionInicial)
print(f"Sus probabilidades de selección fueron:\n{pis}\n")
print(f"El más apto fue:\n{masApto}\n")

qis = Probas_Acumulativas(pis)
print(f"Las probabilidades acumulativas fueron:\n{qis}\n")

parejas = Generar_Parejas(poblacionInicial, qis)
print(f"Las parejas seleccionadas para la reproducción fueron:\n{parejas}\n")

parejasBinarias = Padres_Binarios(parejas, nBits, intervalo)
print(f"Los padres en binario fueron:\n{parejasBinarias}\n")

hijosBinarios = cruza_n_puntos_np_numba(parejasBinarias, nPuntos)
print(f"Los hijos en binario fueron:\n{hijosBinarios}\n")


La población inicial fue:
[[-0.07870473  1.34435016]
 [-1.76531482  1.48475712]
 [ 0.07130104 -0.31841013]
 [ 2.04006772  1.02714325]]

Sus probabilidades de selección fueron:
[0.25000005 0.25000002 0.25000009 0.24999984]

El más apto fue:
[ 0.07130104 -0.31841013]

Las probabilidades acumulativas fueron:
[0.25000005 0.50000007 0.75000016 1.        ]

Las parejas seleccionadas para la reproducción fueron:
[[[-0.07870473  1.34435016]
  [ 2.04006772  1.02714325]]

 [[ 0.07130104 -0.31841013]
  [-1.76531482  1.48475712]]]

Los padres en binario fueron:
[[[[0 1 1 1 0]
   [1 1 0 0 1]]

  [[1 1 1 1 0]
   [1 0 1 1 1]]]


 [[[1 0 0 0 0]
   [0 1 1 0 1]]

  [[0 0 0 1 0]
   [1 1 0 1 0]]]]

Los hijos en binario fueron:
[[[0 1 1 1 0]
  [1 0 0 1 1]]

 [[1 1 1 1 0]
  [1 1 1 0 1]]

 [[1 0 0 1 0]
  [0 1 1 1 1]]

 [[0 0 0 0 0]
  [1 1 0 0 0]]]

